In [1]:
import os, json
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

from langsmith import Client
client = Client()

In [2]:
from langchain.memory import CosmosDBChatMessageHistory

# Get the conversation for a specific user and session.
session_id = "Agent-Test-Session660"
user_id = "Agent-Test-User127"

cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id=session_id,
    user_id=user_id,
)

# Prepare the CosmosDB for the session.
cosmos.prepare_cosmos()

cosmos.messages

[]

In [3]:
def cosmos_messages_to_string(messages: list) -> str:
    """
    Converts a list of messages to a string.
    """
    chat_str = ""
    for message in messages:
        chat_str += "Customer: " + message.content + "\n"
        chat_str += "AI: " + message.content + "\n"
    return chat_str

chat_str = cosmos_messages_to_string(cosmos.messages)
print(chat_str)

In [4]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field, root_validator, ValidationError
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

class OrderedItem(BaseModel):
    """
    Information about each ordered item such as item name, quantity, price and additional notes provided by the customer.
    """
    item: str = Field(description="Name of the item ordered")
    quantity: int = Field(description="Quantity of that ordered item")
    price: float = Field(description="Price of that ordered item")
    item_notes: Optional[str] = Field(description="Any additional notes about that item provided by the customer")
    
class OrderInfo(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract order details such item name, quantity, price, and any additional notes for each item.
    """
    order: List[OrderedItem] = Field(description="List of items ordered and their details")

order_schema = convert_pydantic_to_openai_function(OrderInfo)
print(json.dumps(order_schema, indent=4))

{
    "name": "OrderInfo",
    "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract order details such item name, quantity, price, and any additional notes for each item.",
    "parameters": {
        "$defs": {
            "OrderedItem": {
                "description": "Information about each ordered item such as item name, quantity, price and additional notes provided by the customer.",
                "properties": {
                    "item": {
                        "description": "Name of the item ordered",
                        "title": "Item",
                        "type": "string"
                    },
                    "quantity": {
                        "description": "Quantity of that ordered item",
                        "title": "Quantity",
                        "type": "integer"
                    },
                    "price": {
                        "descript

In [5]:
class CustomerName(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract the customer's name.
    """
    customer_name: str = Field(description="Name of the customer")
    
customer_name_schema = convert_pydantic_to_openai_function(CustomerName)
print(json.dumps(customer_name_schema, indent=4))

{
    "name": "CustomerName",
    "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract the customer's name.",
    "parameters": {
        "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract the customer's name.",
        "properties": {
            "customer_name": {
                "description": "Name of the customer",
                "title": "Customer Name",
                "type": "string"
            }
        },
        "required": [
            "customer_name"
        ],
        "title": "CustomerName",
        "type": "object"
    }
}


In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

model = AzureChatOpenAI(temperature=0,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

extraction_functions = [convert_pydantic_to_openai_function(OrderInfo)]

order_prompt = ChatPromptTemplate.from_messages([
    ("system", """
            You are a world class algorithm for extracting information in structured formats from a restaurant order conversation between a customer and an AI assistant.
            Your task is to extract and save the pertinent details, specifically the ordered food items, quantity ordered, their prices and any additional notes requested for each ordered item included in the order. 
            Make sure to extract only those items that were ordered.
        """),
    ("user", "{input}")
])

extraction_model = model.bind(
    functions=extraction_functions,
    function_call={"name": "OrderInfo"}
)

order_extraction_chain = order_prompt | extraction_model | JsonOutputFunctionsParser()
order_result = order_extraction_chain.invoke({"input": chat_str})
print(json.dumps(order_result, indent=4))


{
    "order": [
        {
            "item": "Burger",
            "quantity": 2,
            "price": 10,
            "item_notes": "No onions"
        },
        {
            "item": "Fries",
            "quantity": 1,
            "price": 5,
            "item_notes": null
        },
        {
            "item": "Coke",
            "quantity": 3,
            "price": 2,
            "item_notes": null
        }
    ]
}


In [7]:
extraction_functions = [convert_pydantic_to_openai_function(CustomerName)]

name_prompt = ChatPromptTemplate.from_messages([
    ("system", """
            You are a world class algorithm for extracting information in structured formats from a restaurant order conversation between a customer and an AI assistant.
            Your task is to extract the customer's name included in the order.
        """),
    ("user", "{input}")
])

extraction_model = model.bind(
    functions=extraction_functions,
    function_call={"name": "CustomerName"}
)

name_extraction_chain = name_prompt | extraction_model | JsonOutputFunctionsParser()
name = name_extraction_chain.invoke({"input": chat_str})
customer_name = name["customer_name"]
print(f"Customer name: {customer_name}")

Customer name: John
